# PMAC: Historical Comparisons

In [1]:
import pandas as pd

from IPython.display import HTML

import pmac_utils
from shared_utils import rt_dates

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
dfs = {}
for key, date in rt_dates.PMAC.items():
    df = pmac_utils.flag_parallel_intersecting_routes(date)
    
    # In D2_pmac, only keep subset where _merge=="both"
    # If it's not able to merge correctly at itp_id-route_id level, let's drop those
    plot_df = df[df._merge=="both"]
    summary = pmac_utils.get_summary_table(plot_df) 
    dfs[key] = summary

In [4]:
for date, dataset in dfs.items():
    print(f"{date}: {dataset.total_service_hours.sum()}")

Q1_2022: 91329
Q2_2022: 109391


In [5]:
def change_from_prior(current: pd.DataFrame, prior: pd.DataFrame, 
                      col: str) -> pd.DataFrame:
    keep_cols = ["category", col]
    
    current2 = current[keep_cols].rename(columns={col: "current"})
    prior2 = prior[keep_cols].rename(columns={col: "prior"})
                  
    df = pd.merge(current2, prior2, on = "category", how = "inner")
    
    df = df.assign(
        change = df.current - df.prior,
        pct_change = (df.current - df.prior).divide(df.prior)
    )
        
    df_style = (pmac_utils.sort_by_column(df)
           .style.format(
               subset=['current', 'prior', 'change'], 
               **{'formatter': '{:,}'})
                .format(
                    subset=['pct_change'],
                    **{'formatter': '{:,.3f}'}
                )
                .set_properties(
                    subset=['current', 'prior', 'change', 'pct_change'], 
                    **{'text-align': 'center'})
                .set_properties(
                    subset='category',
                    **{'text-align': 'left'}
                )
                .set_table_styles([dict(selector='th', 
                                        props=[('text-align', 'center')])
                                        ])
           .hide(axis="index")
           .to_html()
          )
    
    display(HTML(df_style))

In [6]:
change_from_prior(dfs["Q2_2022"], dfs["Q1_2022"], "total_service_hours")

category,current,prior,change,pct_change
parallel,"59,465","45,860","13,605",0.297
on_shn,"19,527","13,644","5,883",0.431
other,"30,399","31,825","-1,426",-0.045
